In [11]:
%pip install pyarrow

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 1.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [17]:
import pandas as pd
import numpy as np 
import re
from transformers import BertModel, BertTokenizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
import sklearn.metrics as metrics

In [26]:
data = pd.read_csv('slide_info.csv')

words_to_remove = ['Saake','Database', 'Concepts', 'Last', 'Edited:', 'April', '2022']

def remove_words(text):
    for word in words_to_remove:
        text = text.replace(word, '')
    return text

data['Content'] = data['Content'].apply(lambda x: remove_words(x))
#data['Content'] = data['Content'].apply(lambda x: x.lower())
#data['Content'] = data['Content'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '',x))



In [19]:

def get_bert_encodings(x):
    model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')
    embeddings = model.encode(x)
    return embeddings

data['encodings'] = data['Content'].apply(lambda x: get_bert_encodings(x))

In [24]:
def preprocess_text(text):
  text = text.lower()
  text = re.sub('[^A-Za-z0-9]+', ' ', text)
  return text

#keywords = 'The update Statement Syntax: update base_relationset attribute1 = expression1attributen = expressionn[ where condition ]'
#keywords = 'Selection: Choose rows in a table based on a selection predicate σVintage>2000(WINES)'
Keywords = 'select * from WINES where Name = ’La Rose’ or Name = ’La RoseA’ or Name = ’La RoseAA’ .. or Name = ’La RoseB’ or Name = ’La RoseBB’ .. ... or Name = ’La Rose Grand Cru’ ... or Name = ’La Rose Grand Cru Classe’ ... ... or Name = ’La RoseZZZZZZZZZZZZZ’'
#keywords = 'Selection: Choose rows in a table based on a selection predicate σVintage>2000(WINES) WineID Name Color Vintage Vineyard2168 Creek Shiraz Red 2003 Creek3456 Zinfandel Red 2004 Helena2171 Pinot Noir Red 2001 Creek4961 Chardonnay White 2002 Bighorn'
processed_keywords = preprocess_text(keywords)
keyword_encodings = get_bert_encodings(processed_keywords)


In [25]:

keyword_encodings = keyword_encodings.reshape(1, -1)
data['similarity_score'] = data['encodings'].apply(lambda x: metrics.pairwise.cosine_similarity(x.reshape(1,-1), keyword_encodings)[0][0])
df_results = data.sort_values(by=['similarity_score'], ascending=False)
df_results 

,Slide Number,Content,encodings,similarity_score
25,"PDF-2, Pg26",relational s data as tables basic operations ...,"[-0.8499671, 0.73725426, -0.31160977, -0.07073...",0.945626
44,"PDF-2, Pg45",relational s data as tables manipulation oper...,"[-0.6675462, 0.56670976, -0.5475144, -0.034006...",0.923235
10,"PDF-2, Pg11",relational s data as tables relations for tab...,"[-0.8389369, 0.61941844, -0.45466134, -0.06680...",0.921224
43,"PDF-2, Pg44",relational s data as tables manipulation oper...,"[-0.7042547, 0.6840975, -0.744903, 0.051786743...",0.919385
14,"PDF-2, Pg15",relational s data as tables relations for tab...,"[-0.9684211, 0.37535018, -0.597448, -0.0564034...",0.907441
...,...,...,...,...
58,"PDF-6, Pg1",part vi\nthe relational query language sql,"[-0.25353575, 0.909323, 0.826501, 0.16766506, ...",0.323136
144,"PDF-6, Pg87",the relational query language sql special join...,"[-0.21429957, 1.0940974, -0.30548704, 0.290438...",0.311410
33,"PDF-2, Pg34",relational s data as tables basic operations ...,"[-0.83020616, 1.0787034, 0.40658092, 0.0031655...",0.306868
20,"PDF-2, Pg21",relational s data as tables sql data denition...,"[-0.10148697, 1.4401929, -0.33007684, -0.10712...",0.299067
